In [1]:
import polars as pl
from pathlib import Path

In [3]:
meta_df = pl.read_csv("data/movies_metadata.csv", infer_schema_length=10000000)

In [4]:
meta_df

adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,f64,str,str,str,str,bool,f64,i64
"""False""","""{'id': 10194, …","""30000000""","""[{'id': 16, 'n…","""http://toystor…","""862""","""tt0114709""","""en""","""Toy Story""","""Led by Woody, …","""21.946943""","""/rhIRbceoE9lR4…","""[{'name': 'Pix…","""[{'iso_3166_1'…","""1995-10-30""",373554033,81.0,"""[{'iso_639_1':…","""Released""",null,"""Toy Story""",false,7.7,5415
"""False""",null,"""65000000""","""[{'id': 12, 'n…",null,"""8844""","""tt0113497""","""en""","""Jumanji""","""When siblings …","""17.015539""","""/vzmL6fP7aPKNK…","""[{'name': 'Tri…","""[{'iso_3166_1'…","""1995-12-15""",262797249,104.0,"""[{'iso_639_1':…","""Released""","""Roll the dice …","""Jumanji""",false,6.9,2413
"""False""","""{'id': 119050,…","""0""","""[{'id': 10749,…",null,"""15602""","""tt0113228""","""en""","""Grumpier Old M…","""A family weddi…","""11.7129""","""/6ksm1sjKMFLbO…","""[{'name': 'War…","""[{'iso_3166_1'…","""1995-12-22""",0,101.0,"""[{'iso_639_1':…","""Released""","""Still Yelling.…","""Grumpier Old M…",false,6.5,92
"""False""",null,"""16000000""","""[{'id': 35, 'n…",null,"""31357""","""tt0114885""","""en""","""Waiting to Exh…","""Cheated on, mi…","""3.859495""","""/16XOMpEaLWkrc…","""[{'name': 'Twe…","""[{'iso_3166_1'…","""1995-12-22""",81452156,127.0,"""[{'iso_639_1':…","""Released""","""Friends are th…","""Waiting to Exh…",false,6.1,34
"""False""","""{'id': 96871, …","""0""","""[{'id': 35, 'n…",null,"""11862""","""tt0113041""","""en""","""Father of the …","""Just when Geor…","""8.387519""","""/e64sOI48hQXyr…","""[{'name': 'San…","""[{'iso_3166_1'…","""1995-02-10""",76578911,106.0,"""[{'iso_639_1':…","""Released""","""Just When His …","""Father of the …",false,5.7,173
"""False""",null,"""60000000""","""[{'id': 28, 'n…",null,"""949""","""tt0113277""","""en""","""Heat""","""Obsessive mast…","""17.924927""","""/zMyfPUelumio3…","""[{'name': 'Reg…","""[{'iso_3166_1'…","""1995-12-15""",187436818,170.0,"""[{'iso_639_1':…","""Released""","""A Los Angeles …","""Heat""",false,7.7,1886
"""False""",null,"""58000000""","""[{'id': 35, 'n…",null,"""11860""","""tt0114319""","""en""","""Sabrina""","""An ugly duckli…","""6.677277""","""/jQh15y5YB7bWz…","""[{'name': 'Par…","""[{'iso_3166_1'…","""1995-12-15""",0,127.0,"""[{'iso_639_1':…","""Released""","""You are cordia…","""Sabrina""",false,6.2,141
"""False""",null,"""0""","""[{'id': 28, 'n…",null,"""45325""","""tt0112302""","""en""","""Tom and Huck""","""A mischievous …","""2.561161""","""/sGO5Qa55p7wTu…","""[{'name': 'Wal…","""[{'iso_3166_1'…","""1995-12-22""",0,97.0,"""[{'iso_639_1':…","""Released""","""The Original B…","""Tom and Huck""",false,5.4,45
"""False""",null,"""35000000""","""[{'id': 28, 'n…",null,"""9091""","""tt0114576""","""en""","""Sudden Death""","""International …","""5.23158""","""/eoWvKD60lT95S…","""[{'name': 'Uni…","""[{'iso_3166_1'…","""1995-12-22""",64350171,106.0,"""[{'iso_639_1':…","""Released""","""Terror goes in…","""Sudden Death""",false,5.5,174


In [9]:
meta_df = (
    meta_df.with_columns(
        adult=pl.col("adult") == "True",
        budget=pl.col("budget").str.parse_int(10, strict=False),
        id=pl.col("id").str.parse_int(10, strict=False),
        imdb_id=pl.col("imdb_id").str.replace("tt", "").str.parse_int(10).cast(pl.Int64),
        popularity=pl.col("popularity").cast(pl.Float64, strict=False),
        release_date=pl.col("release_date").str.to_date(format="%Y-%m-%d", strict=False),
        status=pl.col("status").cast(pl.Categorical),
    )
)

In [10]:
movie_df = meta_df.select(
    [
        "id",
        "imdb_id",
        "original_title",
        "popularity",
        "release_date",
        "revenue",
        "runtime",
        "vote_average",
        "vote_count",
    ]
)

In [13]:
movie_df.write_parquet(processed_folder / "movie.parquet")

In [14]:
movie_df = pl.read_parquet(processed_folder / "movie.parquet")
links_df = pl.read_csv("data/links.csv")
review_df = pl.read_csv("data/ratings.csv")

In [15]:
(
    review_df
    .join(links_df, on="movieId", how="inner")
    .join(movie_df, left_on="imdbId", right_on="imdb_id", how="inner")
    .write_parquet(processed_folder / "processed.parquet")
)